In [1]:
import csv

In [62]:
f = open("gab.tsv")
gab_reader = csv.reader(f, delimiter='\t')
for row in gab_reader:
    headers_list = row
    break

In [63]:
gab_list = list()
for row in gab_reader:
    gab = dict()
    for i, header in enumerate(headers_list):
        gab[header] = row[i]
    gab_list.append(gab)

In [64]:
len(gab_list)

86529

In [65]:
headers = headers_list

In [66]:
for gab in gab_list:
    for i in range(3,17):
        if gab[headers_list[i]]=='0' or gab[headers_list[i]]=='' or gab[headers_list[i]]=='0.0' or gab[headers_list[i]]==0:
            gab[headers_list[i]]=0
        elif gab[headers_list[i]]=='1' or gab[headers_list[i]]=='1.0' or gab[headers_list[i]]==1:
            gab[headers_list[i]]=1
        else:
            if gab[headers_list[i]].startswith('0'):
                gab[headers_list[i]]=0
            elif gab[headers_list[i]].startswith('1'):
                gab[headers_list[i]]=1
            else:
                print(gab)

In [67]:
gab_list[0]

{'ID': '27044',
 'Annotator': '4',
 'Text': 'Ah the PSYOPS antifa crew is back. That’s how I know we are winning.',
 'Hate': 0,
 'HD': 0,
 'CV': 0,
 'VO': 0,
 'REL': 0,
 'RAE': 0,
 'SXO': 0,
 'GEN': 0,
 'IDL': 0,
 'NAT': 0,
 'POL': 0,
 'MPH': 0,
 'EX': 0,
 'IM': 0}

In [68]:
# gab preprocess
anno_cnt = 0
cur_id = 0
gab_corpus = list()
for gab in gab_list:
    if cur_id == 0:
        cur_gab = gab
        cur_id = gab['ID']
        continue
    if cur_id != gab['ID']:
        for i in range(3, 17):
            if cur_gab[headers[i]] / anno_cnt < 0.5:
                cur_gab[headers[i]] = 0
            else:
                cur_gab[headers[i]] = 1
        del cur_gab['Annotator']
        del cur_gab['Hate']
        cur_gab['text_id'] = cur_gab.pop('ID')
        cur_gab['im'] = cur_gab.pop('IM')
        cur_gab['hd'] = cur_gab.pop('HD')
        cur_gab['cv'] = cur_gab.pop('CV')
        cur_gab['vo'] = cur_gab.pop('VO')
        cur_gab['rel'] = cur_gab.pop('REL')
        cur_gab['rae'] = cur_gab.pop('RAE')
        cur_gab['sxo'] = cur_gab.pop('SXO')
        cur_gab['gen'] = cur_gab.pop('GEN')
        cur_gab['idl'] = cur_gab.pop('IDL')
        cur_gab['nat'] = cur_gab.pop('NAT')
        cur_gab['pol'] = cur_gab.pop('POL')
        cur_gab['mph'] = cur_gab.pop('MPH')
        cur_gab['ex'] = cur_gab.pop('EX') 
        gab_corpus.append(cur_gab)
        cur_id = gab['ID']
        anno_cnt = 1
        cur_gab = gab
    else:
        for i in range(3,17):
            try:
                cur_gab[headers[i]] += gab[headers[i]]
            except:
                print("cur", type(cur_gab[headers[i]]))
                print("gab", type(gab[headers[i]]))
        anno_cnt += 1
    

In [69]:
len(gab_corpus)

27664

In [77]:
gab_corpus[0]

{'Text': 'Ah the PSYOPS antifa crew is back. That’s how I know we are winning.',
 'text_id': '27044',
 'im': 0,
 'hd': 0,
 'cv': 0,
 'vo': 0,
 'rel': 0,
 'rae': 0,
 'sxo': 0,
 'gen': 0,
 'idl': 0,
 'nat': 0,
 'pol': 0,
 'mph': 0,
 'ex': 0}

In [76]:
total_num = len(gab_corpus)
val_num = int(total_num * 0.05)
test_num = int(total_num * 0.05)
train_num = total_num - val_num - test_num

In [79]:
import random
ghc = gab_corpus
random.shuffle(ghc)

In [82]:
ghc_val = ghc[:val_num]
ghc_test = ghc[val_num:val_num+test_num]
ghc_train = ghc[val_num+test_num:]

In [74]:
import json
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

In [83]:
dump_jsonl(ghc_train, 'train.jsonl')
dump_jsonl(ghc_val, 'val.jsonl')
dump_jsonl(ghc_test, 'test.jsonl')

Wrote 24898 records to train.jsonl
Wrote 1383 records to val.jsonl
Wrote 1383 records to test.jsonl
